*   Make sure you run this using a GPU (On Google Colab: Runtime -> Change Runtime Type -> GPU)

# Setup

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
!pip install -q ase
!pip install -q torch==1.8.0
!pip install  torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git@1.7.0

     |████████████████████████████████| 2.2 MB 5.2 MB/s 
     |████████████████████████████████| 735.5 MB 12 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0 which is incompatible.
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
     |████████████████████████████████| 8.1 MB 2.6 MB/s 
     |████████████████████████████████| 3.1 MB 2.5 MB/s eta 0:00:01

In [ ]:
import numpy as np
import torch
import ase
import random
from torch_geometric.data import Data, DataLoader
from ase.io import read

In [ ]:
SEED = 55555

np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

## Set training hyperparameters

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # use GPU if available
DTYPE = torch.float64  # data type to use for data and model

BASE_PATH = '/content/drive/MyDrive/gnn_atomistics/'  # path to git repo/colab/gnn_atomistics

TRAINING_RATIO = 0.8  # percent of the dataset to use for training
OPTIMIZER = torch.optim.Adam  
BATCH_SIZE = 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = BASE_PATH + 'data/training/'
MODELS_PATH = BASE_PATH + "/models"

# Data

## Define functions

In [ ]:
M = {}
def extend_atoms(atoms, source, target):
  global M
  if source not in M.keys():
    M[source] = {}
  if target not in M[source].keys():
    M[source][target] = ase.build.find_optimal_cell_shape(atoms.get_cell(), target, "sc") 
  supercell = ase.build.make_supercell(atoms, M[source][target])
  supercell.info["energy"] = atoms.info["energy"] * int(target / source)
  supercell.info["virial"] = atoms.info["virial"] * int(target / source)
  return supercell

In [ ]:
def data_object(atoms: ase.Atoms, db = None, device = DEVICE):

  n = atoms.get_global_number_of_atoms()
  if n == 1:
    atoms = extend_atoms(atoms, 1, 54)
    n = atoms.get_global_number_of_atoms()

  cell = torch.tensor(atoms.cell, dtype=DTYPE, device=device)
  x = torch.tensor(atoms.get_positions(), dtype=DTYPE, device=device)
  z = torch.tensor(atoms.get_array("numbers", copy=True), dtype=torch.long, device=device)
  y = torch.tensor(atoms.info["energy"], dtype=DTYPE, device=device)
  f = torch.tensor(atoms.get_array("force", copy=True), dtype=DTYPE, device=device)
  if db == 1:
    v = torch.tensor(atoms.info["virial"], dtype=DTYPE, device=device)
  else:
    v = torch.zeros((3,3), dtype=DTYPE, device=device)
  db = torch.tensor(db, dtype=torch.long, device=device)

  return Data(z=z, x=x, cell=cell, y=y, f=f, v=v, n=n, db=db)

## Load data

In [ ]:
data = []
for i in range(1,9):
  fn = f"{DATA_PATH}/DB{i}.xyz"
  db = read(fn, index=":")
  data += [data_object(a, db = i, device = "cpu") for a in db]
print(len(data))

In [ ]:
print(data[0])

In [ ]:
y_atom = torch.tensor([d.y / d.n for d in data], dtype=DTYPE)
y_atom_mean = y_atom.mean()
y_atom_std = y_atom.std()
print(y_atom_mean, y_atom_std)

In [ ]:
v_atom = torch.stack([d.v / d.n for d in data if d.db == torch.tensor(1)])
v_atom_mean = v_atom.mean(dim=0)
v_atom_std = v_atom.std(dim=0)
print(v_atom_mean)
print(v_atom_std)

In [ ]:
mean = torch.cat((y_atom_mean.unsqueeze(0), torch.flatten(v_atom_mean))).to(DEVICE)
std = torch.cat((y_atom_std.unsqueeze(0), torch.flatten(v_atom_std))).to(DEVICE)
print(mean)
print(std)

In [ ]:
random.shuffle(data)
train_amount = int(len(data) * TRAINING_RATIO)
train_data = data[:train_amount]
test_data = data[train_amount:]
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# Model


In [ ]:
#@title  { vertical-output: true, form-width: "25%" }
from torch_geometric.nn import DimeNet
from torch_geometric.nn.acts import swish
from math import sqrt, pi as PI

import numpy as np
import torch
from torch.nn import Linear, Embedding
from torch_scatter import scatter
from torch_sparse import SparseTensor
from torch_geometric.nn import radius_graph
from torch_geometric.data import download_url
from torch_geometric.data.makedirs import makedirs

from torch_geometric.nn.models.dimenet import Envelope
from torch_geometric.nn.models.dimenet import BesselBasisLayer
from torch_geometric.nn.models.dimenet import SphericalBasisLayer
from torch_geometric.nn.models.dimenet import ResidualLayer
from torch_geometric.nn.models.dimenet import InteractionBlock
from torch_geometric.nn.models.dimenet import OutputBlock

from torch_geometric.nn.models.dimenet_utils import bessel_basis, real_sph_harm
import ase
from ase.neighborlist import neighbor_list 
from ase import Atoms

from torch_geometric.data import DataLoader

try:
    import sympy as sym
except ImportError:
    sym = None

import os
try:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    import tensorflow as tf
except ImportError:
    tf = None

# TODO: move this somewhere else
device = DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.float64


def pbc_edges(cutoff, z, x, cell, batch, compute_sc=False):

  NH1 = torch.tensor([], dtype=torch.long, device=DEVICE)
  NH2 = torch.tensor([], dtype=torch.long, device=DEVICE)
  S = torch.tensor([], dtype=torch.long, device=DEVICE)
  D = torch.tensor([], dtype=DTYPE, device=DEVICE)
  SC = torch.tensor([], dtype=DTYPE, device=DEVICE) if compute_sc else None
  x_ = torch.clone(x).detach().cpu().numpy()

  if batch is not None:
    # count number of elements for each batch
    batch_ids = list(set(batch.cpu().tolist()))
    batch_sizes = [ (batch == id).sum().item() for id in batch_ids ]

    for i in range(len(batch_sizes)):
      offset = sum(batch_sizes[:i]) # to obtain correct atom indices
      
      atoms = Atoms(charges = (z[offset:offset + batch_sizes[i]]).cpu(), 
        positions = x_[offset:offset + batch_sizes[i]], 
        cell = (cell[3*i:3*(i+1)]).cpu(),
        pbc=True
      ) 
      
      nh1, nh2, s = neighbor_list("ijS", atoms, cutoff, self_interaction=False) 
      nh1 = torch.tensor(nh1, dtype=torch.long, device=DEVICE)
      nh2 = torch.tensor(nh2, dtype=torch.long, device=DEVICE)
      nh1 = nh1 + offset
      nh2 = nh2 + offset
      s = torch.tensor(s, dtype=DTYPE, device=DEVICE)
      d = x[nh2] - x[nh1] + torch.matmul(s, cell[3*i:3*(i+1)])
      
      if compute_sc:
        cell_flat = torch.flatten(cell[3*i:3*(i+1)])
        sc = torch.tile(cell_flat, (len(d), 1))
        sc[:, 0:3] = (sc[:, 0:3].T * s[:, 0]).T
        sc[:, 3:6] = (sc[:, 3:6].T * s[:, 1]).T
        sc[:, 6:9] = (sc[:, 6:9].T * s[:, 2]).T
        SC = torch.cat((SC, sc), 0)      

      NH1 = torch.cat((NH1, nh1), 0)
      NH2 = torch.cat((NH2, nh2), 0)
      S = torch.cat((S, s), 0)
      D = torch.cat((D, d), 0)

  else: # no batch
    atoms = Atoms(charges = z.cpu(), positions = x.detach().numpy(), cell = cell.cpu(), pbc=True)
    nh1, nh2, s = neighbor_list("ijS", atoms, cutoff, self_interaction=False)
    nh1 = torch.tensor(nh1, dtype=torch.long, device=DEVICE)
    nh2 = torch.tensor(nh2, dtype=torch.long, device=DEVICE)
    s = torch.tensor(s, dtype=DTYPE, device=DEVICE)
    d = x[nh2] - x[nh1] + torch.matmul(s, cell)
    
    if compute_sc:
      cell_flat = torch.flatten(cell)
      sc = torch.tile(cell_flat, (len(d), 1))
      sc[:, 0:3] = (sc[:, 0:3].T * s[:, 0]).T
      sc[:, 3:6] = (sc[:, 3:6].T * s[:, 1]).T
      sc[:, 6:9] = (sc[:, 6:9].T * s[:, 2]).T  
      SC = sc

    NH1, NH2, S, D = nh1, nh2, s, d

  D = D.norm(dim=-1)
  return  NH1, NH2, D, S, SC 

class EmbeddingBlock(torch.nn.Module):
  def __init__(self, num_radial, hidden_channels, act=swish):
    super(EmbeddingBlock, self).__init__()
    self.act = act

    self.emb = Embedding(95, hidden_channels)
    self.lin_rbf = Linear(num_radial, hidden_channels, bias=False)
    self.lin = Linear(3 * hidden_channels, hidden_channels)

    self.reset_parameters()

  def reset_parameters(self):
    self.emb.weight.data.uniform_(-sqrt(3), sqrt(3))
    self.lin_rbf.reset_parameters()
    self.lin.reset_parameters()

  def forward(self, x, rbf, i, j):
    x = self.emb(x)
    #rbf = self.act(self.lin_rbf(rbf)) # FIX: this should not have an activation function
    rbf = self.lin_rbf(rbf)
    return self.act(self.lin(torch.cat([x[i], x[j], rbf], dim=-1)))



class DimeNetx(torch.nn.Module):
  
  def __init__(self, hidden_channels, out_channels, num_blocks, num_bilinear,
                 num_spherical, num_radial, cutoff=5.0, envelope_exponent=5,
                 num_before_skip=1, num_after_skip=2, num_output_layers=3,
                 act=swish, mean=None, std=None):
    super(DimeNetx, self).__init__()

    self.cutoff = cutoff

    #set mean and standard deviation of energies
    self.mean = mean 
    self.std = std

    # padding used for PBCs
    self.padding = torch.nn.ConstantPad2d((0,6,0,0), 0)

    if sym is None:
        raise ImportError('Package `sympy` could not be found.')

    self.num_blocks = num_blocks

    self.rbf = BesselBasisLayer(num_radial, cutoff, envelope_exponent)
    self.sbf = SphericalBasisLayer(num_spherical, num_radial, cutoff,
                                    envelope_exponent)

    self.emb = EmbeddingBlock(num_radial, hidden_channels, act)

    self.output_blocks = torch.nn.ModuleList([
        OutputBlock(num_radial, hidden_channels, out_channels,
                    num_output_layers, act) for _ in range(num_blocks + 1)
    ])

    self.interaction_blocks = torch.nn.ModuleList([
        InteractionBlock(hidden_channels, num_bilinear, num_spherical,
                          num_radial, num_before_skip, num_after_skip, act)
        for _ in range(num_blocks)
    ])

    self.reset_parameters()

  def reset_parameters(self):
    self.rbf.reset_parameters()
    self.emb.reset_parameters()
    for out in self.output_blocks:
      out.reset_parameters()
    for interaction in self.interaction_blocks:
      interaction.reset_parameters()

  def triplets_original(self, edge_index, num_nodes):
    row, col = edge_index  # j->i

    value = torch.arange(row.size(0), device=row.device)
    adj_t = SparseTensor(row=col, col=row, value=value,
                         sparse_sizes=(num_nodes, num_nodes))
    adj_t_row = adj_t[row]
    num_triplets = adj_t_row.set_value(None).sum(dim=1).to(torch.long)

    # Node indices (k->j->i) for triplets.
    idx_i = col.repeat_interleave(num_triplets)
    idx_j = row.repeat_interleave(num_triplets)
    idx_k = adj_t_row.storage.col()
    mask = (idx_i != idx_k)  # Remove i == k triplets.
    idx_i, idx_j, idx_k = idx_i[mask], idx_j[mask], idx_k[mask]

    # Edge indices (k-j, j->i) for triplets.
    idx_kj = adj_t_row.storage.value()[mask]
    idx_ji = adj_t_row.storage.row()[mask]

    return col, row, idx_i, idx_j, idx_k, idx_kj, idx_ji


  def triplets(self, edge_index, num_nodes, shift_cells=None, shift=None):
    row, col = edge_index  # j->i

    value = torch.arange(row.size(0), device=row.device)
    adj_t = SparseTensor(row=col, col=row, value=value,
                         sparse_sizes=(num_nodes, num_nodes))
    adj_t_row = adj_t[row]
    num_triplets = adj_t_row.set_value(None).sum(dim=1).to(torch.long)

    idx_i = col.repeat_interleave(num_triplets)
    idx_j = row.repeat_interleave(num_triplets)
    idx_k = adj_t_row.storage.col()

    if shift_cells is not None: # Update also the shift vectors
      shift_cells_i = shift_cells.repeat_interleave(num_triplets, dim=0)
      shift_i = shift.repeat_interleave(num_triplets, dim=0)
      shift_cells_k = -shift_cells[adj_t_row.storage.value()]
      shift_k = -shift[adj_t_row.storage.value()]

    mask = torch.all((torch.cat((torch.unsqueeze(idx_i, 1), shift_i), dim=1) ==\
                      torch.cat((torch.unsqueeze(idx_k, 1), shift_k), dim=1)), dim=1)

    idx_i, idx_j, idx_k = idx_i[~mask], idx_j[~mask], idx_k[~mask]
    if shift_cells is not None: # Remove also from the shift vector
      shift_cells_i = shift_cells_i[~mask]
      shift_cells_k = shift_cells_k[~mask]
      shift_i = shift_i[~mask]
      shift_k = shift_k[~mask]

    idx_kj = adj_t_row.storage.value()[~mask]
    idx_ji = adj_t_row.storage.row()[~mask]

    return col, row, idx_i, idx_j, idx_k, idx_kj, idx_ji, shift_cells_i, shift_i, shift_cells_k, shift_k


  def forward(self, z, pos, cell=None, batch=None):
        
    edge_index = []
    dist = []
    shift_cells = None
    if cell is not None: # implement PBC
      r1, r2, dist, shift, shift_cells = pbc_edges(self.cutoff, z, pos, cell, batch, compute_sc=True) 
      edge_index = [r1, r2]

        
      i, j, idx_i, idx_j, idx_k, idx_kj, idx_ji, shift_cells_i, shift_i, shift_cells_k, shift_k = self.triplets(
          edge_index, num_nodes=z.size(0), shift_cells=shift_cells, shift=shift)        
    else: # old method without PBC
      edge_index = radius_graph(pos, r=self.cutoff, batch=batch)
      i, j, idx_i, idx_j, idx_k, idx_kj, idx_ji = self.triplets_original(
          edge_index, num_nodes=z.size(0))        
      dist = (pos[i] - pos[j]).pow(2).sum(dim=-1).sqrt()

    # Define atoms position 
    pos_i = pos[idx_i]
    pos_j = pos[idx_j]
    pos_k = pos[idx_k]

    if cell is not None: # Fix coordinates for PBCs

      pos_i = pos_i + shift_cells_i[:, 0:3] + shift_cells_i[:, 3:6] + shift_cells_i[:, 6:9]
      pos_k = pos_k + shift_cells_k[:, 0:3] + shift_cells_k[:, 3:6] + shift_cells_k[:, 6:9]
      sc_ij = torch.all(~torch.all(pos_i == pos_j, dim=1)) 
      sc_kj = torch.all(~torch.all(pos_k == pos_j, dim=1))
      #if not (sc_ij and sc_kj):
      #   raise NameError('Found same position for different atoms!')

    # Calculate angles - with some Fixes to indexes compared to the orig. version
    pos_ji, pos_kj = pos_j - pos_i, pos_k - pos_j
    a = (pos_ji * pos_kj).sum(dim=-1)
    b = torch.cross(pos_ji, pos_kj)
    b = torch.linalg.norm(b + 1e-16, dim=-1)  # following https://github.com/pytorch/pytorch/issues/2421
    angle = torch.atan2(b, a)

    rbf = self.rbf(dist)
    sbf = self.sbf(dist, angle, idx_kj)

    # Embedding block.
    x = self.emb(z, rbf, i, j)
    P = self.output_blocks[0](x, rbf, i, num_nodes=pos.size(0))

    # Interaction blocks.
    for interaction_block, output_block in zip(self.interaction_blocks,
                                               self.output_blocks[1:]):
      x = interaction_block(x, rbf, sbf, idx_kj, idx_ji)
      a = output_block(x, rbf, i, num_nodes=pos.size(0))
      P += a

    # Energy de-standardization
    if self.std is not None and self.mean is not None:
      P = P * self.std + self.mean

    res = P.sum(dim=0) if batch is None else scatter(P, batch, dim=0)
    return res

# Training

## Define functions


In [ ]:
def loss_energy_forces_virial(data, e, f, v, c_e, c_f, c_v):
  
  d_e, d_f, d_v = data.y.to(DEVICE), data.f.to(DEVICE), data.v.to(DEVICE)

  acc = 0
  batched_db = torch.tensor([], dtype=torch.long)
  for i in range(data.num_graphs):
    db = data.get_example(i).db
    batched_db = torch.cat((batched_db, db.repeat(3))) 
  db1_mask = (batched_db == 1)
  db1_mask = db1_mask.to(DEVICE)

  v_ok = v[db1_mask]
  if len(v_ok) != 0:
    loss_v = torch.mean(torch.abs(v_ok - d_v[db1_mask])) 
  else:
    loss_v = torch.tensor(0)
  loss_e = torch.mean(torch.abs(e - d_e))
  loss_f = torch.mean(torch.abs(f - d_f))
  loss = c_e * loss_e + c_f * loss_f + c_v * loss_v
  return loss_e, loss_f, loss_v, loss  

In [ ]:
def train_energy_forces_virial(model, loader, optimizer, c_e, c_f, c_v):
  model.train()
  total_loss = total_loss_e = total_loss_f = total_loss_v = 0

  for data in loader:
    z, x, cell, batch = data.z.to(DEVICE), data.x.to(DEVICE), data.cell.to(DEVICE), data.batch.to(DEVICE)
    x.requires_grad = True 
    optimizer.zero_grad()
    
    pred = model(z, x, cell, batch)
    e = pred[:,:1]
    f = -1 * torch.autograd.grad(e, x, grad_outputs=torch.ones_like(e), create_graph=True, retain_graph=True)[0]
    e = e.squeeze(1)
    v = pred[:,1:].reshape(-1, 3)

    loss_e, loss_f, loss_v, loss = loss_energy_forces_virial(data, e, f, v, c_e, c_f, c_v)
    total_loss_e += loss_e.item()
    total_loss_f += loss_f.item()
    total_loss_v += loss_v.item()
    total_loss += loss.item()

    loss.backward()
    optimizer.step()
  print("Total training loss\t e: {}, f: {}, v: {}, all (scaled): {}".format(total_loss_e, total_loss_f, total_loss_v, total_loss))

In [ ]:
def test(model, train_loader, test_loader):
  model.eval()

  with torch.no_grad():
    train_errs = torch.tensor([], dtype=DTYPE)
    test_errs = torch.tensor([], dtype=DTYPE)
  
    for data in train_loader:  # Iterate in batches over the training/test dataset.
      z, x, cell, batch, d_e = data.z.to(DEVICE), data.x.to(DEVICE), data.cell.to(DEVICE), data.batch.to(DEVICE)
      e = model(data.z, data.x, data.cell, data.batch)
      e = e.squeeze(1)
      errs = torch.abs(e.view(-1).cpu() - data.y.cpu())
      train_errs = torch.cat((train_errs, errs))
    train_mae = torch.mean(train_errs).item()

    for data in test_loader:  # Iterate in batches over the training/test dataset.
      z, x, cell, batch = data.z.to(DEVICE), data.x.to(DEVICE), data.cell.to(DEVICE), data.batch.to(DEVICE)
      e = model(data.z, data.x, data.cell, data.batch)
      e = e.squeeze(1)
      errs = torch.abs(e.view(-1).cpu() - data.y.cpu())
      test_errs = torch.cat((test_errs, errs))
    test_mae = torch.mean(test_errs).item()

  return train_mae, test_mae

In [ ]:
from datetime import datetime

train_maes = []
test_maes = []
def train_and_test(model, train_loader, test_loader, optimizer, scheduler, 
                   energy_coeff, forces_coeff, virial_coeff, 
                   epochs=100, starting_epoch=1, save_every=5,
                   early_stopping_starting_epoch=100, patience=100,
                   name=None, description=None):
  if description == None:
    description = datetime.now().strftime('%H:%M:%S')

  lowest_test_mae = 100.
  epochs_not_improved = 0
  save_model_flag = True
  for epoch in range(starting_epoch, epochs):
    print("")
    print(f"Epoch {epoch} ({datetime.now().strftime('%H:%M:%S')})")
    print(f"")
    train_energy_forces_virial(model, train_loader, optimizer, energy_coeff, forces_coeff, virial_coeff)
    train_mae, test_mae = test(model, train_loader, test_loader)
    train_maes.append(train_mae)
    test_maes.append(test_mae)
    print(f"Energy MAE: train {train_mae} eV, test {test_mae} eV")
    if epoch <= early_stopping_starting_epoch:
      lowest_test_mae = test_mae
      save_model_flag = True
    else:
      print('Early stopping active')
      if test_mae < lowest_test_mae:
        epochs_not_improved = 0
        lowest_test_mae = test_mae
        save_model_flag = True
        print('MAE has decreased, saving the model')
      else:
        epochs_not_improved += 1
        save_model_flag = False
        if epochs_not_improved > patience:
          print('Patience has been finished: end of the training')
          break
        else:
          print('MAE has not decreased for ' + str(epochs_not_improved) + ' epochs, go on without saving the model')
    if save_model_flag == True:
      save_model(name=name, description=description, epoch=epoch, train_err=train_mae, test_err=test_mae)
    scheduler.step(test_mae)

In [ ]:
def save_model(name, description, epoch=-1, train_err=-1, test_err=-1):
  model_data = {
    "desc": description,
    "str": model_str,
    "mean": mean,
    "std": std,
    "state": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scheduler": scheduler.state_dict(),
    "train_err": train_err,
    "test_err": test_err
  }  
  if not epoch == -1:
    epoch_str = "_" + str(epoch)
  else:
    epoch_str = ""
  actual_name = f"{name}{epoch_str}"

  dir = f"{MODELS_PATH}/{name}"
  if not os.path.exists(dir):
    os.makedirs(dir)
    print(f"Created directory {dir}")
  torch.save(model_data, f"{dir}/{actual_name}")
  print(f"Saved {actual_name}")

  if os.path.isfile(f"{dir}/{name}_best"):
    best = torch.load(f"{dir}/{name}_best")
    best_train_err = best["train_err"]
    best_test_err = best["test_err"]
  else:
    best_train_err, best_test_err = 10e18, 10e18
  # if train_err < best_train_err and test_err < best_test_err:
  if test_err < best_test_err:
    torch.save(model_data, f"{dir}/{name}_best")
    print(f"Saved best")

## Instantiate model

In [ ]:
model_name = "dimenet_efv_22set" # name of the folder where checkpoints will be saved
model_description = "DimeNet internal PBC, energy 10 + forces 1000 + virial (DB1) 1"

In [ ]:
model = DimeNetx(hidden_channels=128, out_channels=10, num_blocks=7, num_bilinear=8, num_spherical=7, num_radial=6, cutoff=3.5, mean=mean, std=std).to(DEVICE)

if DTYPE == torch.float64:
  model = model.double()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer.zero_grad()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='abs')
max_epochs = 100
energy_coeff = 10
forces_coeff = 1000
virial_coeff = 1

Copy-paste the content of the previous cell in the following variable to allow for saving of parameters + hyperparameters

In [ ]:
model_str = """
model = DimeNetx(hidden_channels=128, out_channels=10, num_blocks=7, num_bilinear=8, num_spherical=7, num_radial=6, cutoff=3.5, mean=mean, std=std).to(DEVICE)

if DTYPE == torch.float64:
  model = model.double()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer.zero_grad()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='abs')
max_epochs = 100
energy_coeff = 10
forces_coeff = 1000
virial_coeff = 1
"""

# [Restore from checkpoint]

In [ ]:
RESTORE_FROM_CHECKPOINT = False

if RESTORE_FROM_CHECKPOINT:
  model_fn = ""
  checkpoint = torch.load(MODELS_PATH + "/dimenet_last/" + model_fn)
  model.load_state_dict(checkpoint["state"])
  optimizer.load_state_dict(checkpoint["optimizer"])
  scheduler.load_state_dict(checkpoint["scheduler"])

# print(model)
# print(optimizer)
# print(scheduler)

# Info

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
def experiment_summary():
  device = "CPU" if DEVICE == torch.device("cpu") else f"GPU ({torch.cuda.get_device_name(device=DEVICE)})"
  print(f"Device: {device}")
  print(f"Data type: {DTYPE}")
  #print(f"Files used: {file_names}")
  print(f"Cutoff: {3.5}")
  print(f"Mean: {y_atom_mean}, std: {y_atom_std}")
  print(f"Training ratio: {TRAINING_RATIO}")
  print(f"Batch size: {BATCH_SIZE}")
  print(f"Model: \n    {model_name}: {model_description}\n    {str(model)}")
  #loss = "energy" if not use_forces else f"energy+forces ({energy_coeff})"
  #print(f"Loss: {loss}")
  print(f"Optimizer: {optimizer}")
  print(f"Scheduler: {scheduler}")
  print(f"Max epochs: {max_epochs}")
experiment_summary()

Device: GPU (Tesla K80)
Data type: torch.float64
Cutoff: 3.5
Mean: -3460.8258474824015, std: 0.1657638302944937
Training ratio: 0.8
Batch size: 3
Model: 
    dimenet_efv_22set: DimeNet internal PBC, energy 10 + forces 1000 + virial (DB1) 1
    DimeNetx(
  (padding): ConstantPad2d(padding=(0, 6, 0, 0), value=0)
  (rbf): BesselBasisLayer(
    (envelope): Envelope()
  )
  (sbf): SphericalBasisLayer(
    (envelope): Envelope()
  )
  (emb): EmbeddingBlock(
    (emb): Embedding(95, 128)
    (lin_rbf): Linear(in_features=6, out_features=128, bias=False)
    (lin): Linear(in_features=384, out_features=128, bias=True)
  )
  (output_blocks): ModuleList(
    (0): OutputBlock(
      (lin_rbf): Linear(in_features=6, out_features=128, bias=False)
      (lins): ModuleList(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): Linear(in_features=128, out_features=128, bias=True)
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
      (lin): Linear(in_fe

In [ ]:
torch.cuda.get_device_name(device=DEVICE)

'Tesla K80'

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
from psutil import *
print(cpu_count())
print(cpu_stats())
!cat /proc/cpuinfo
!df -h
print(virtual_memory())


2
scpustats(ctx_switches=9175930, interrupts=6448809, soft_interrupts=3856515, syscalls=0)
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alig

In [ ]:
train_and_test(model, train_loader, test_loader, optimizer, scheduler, 
               energy_coeff, forces_coeff, virial_coeff,
               epochs=max_epochs, starting_epoch=1, save_every=1,
               early_stopping_starting_epoch=40, patience=10,
               name=model_name, description=model_description)

In [ ]:
torch.save(train_maes, MODELS_PATH + '/' + model_name + '/train_MAE')
torch.save(test_maes, MODELS_PATH + '/' + model_name + '/test_MAE')

In [ ]:
lista_train_mae = torch.load(MODELS_PATH + '/' + model_name + '/train_MAE')
lista_test_mae = torch.load(MODELS_PATH + '/' + model_name + '/test_MAE')
print(len(lista_train_mae))
print(lista_train_mae)
print(len(lista_test_mae))
print(lista_test_mae)

In [ ]:
# train_and_test(model, train_loader, test_loader, optimizer, scheduler, 
#                use_forces=use_forces, energy_coeff=energy_coeff,
#                epochs=max_epochs, starting_epoch=1, save_every=1,
#                name=model_name, description=model_description)

## [Clean memory]

In [ ]:
import gc
if model:
  model.cpu()
  del model
  gc.collect()
  torch.cuda.empty_cache()